In [1]:
%pylab
import ipywidgets as widgets
from functools import partial
from IPython.display import display
import pandas, time, socket, imp
import OSC,threading

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
sonmod = imp.load_source('sonmod', 'EmoSonics-soundmodels.py')
# I moved it from .py to here to accomodate the non notebook version of rating
print "wait for SC to start..."
time.sleep(5); # wait until SC is started...
print "resume: establish OSC interfaces..."

wait for SC to start...
resume: establish OSC interfaces...


In [4]:
targetlist = ['happy', 'surprised', 'angry', 'afraid', 'neutral', 'disgusted', 
              'sad', 'tired', 'calm'];
degreelist = ['-', 'weak', 'average', 'strong']
sm = 'abstract'
ipAddr = socket.gethostbyname(socket.getfqdn())
v = sonmod.parvecunmap(sonmod.parspecs[sm], 
                   sonmod.parspecs[sm]['default'].tolist())


In [10]:
# study related service functions
def savedata(dataset, prefix="user-soundmodel-run-"):
    df = pandas.DataFrame(data, 
        columns=['uid', 'snd', 'run', 'step', 'time', 'parvec', 'emotion', 'intensity'])
    df.to_csv(prefix + time.strftime("-%Y%m%d-%H%M%S") + ".csv", index=False)
    data.append([userid.value, soundmodel.value, run.value, 
                 count, time.time()-start_time, v, 
                 wRBtarget.value, wRBintensity.value])
    



def mutate(parent, sigma=0.1):
    d=size(parent)
    print parent
    print "size is " + str(d)
    child = clip((parent + sigma*numpy.random.randn(d)), 0, 1)
    return child

def create_next_generation(parentvec, sigma, nr_of_children=4):
    global generation_counter, v
    generation_counter += 1
    return [parentvec] + [mutate(parentvec, sigma) for k in range(nr_of_children)]


In [13]:

data = []
class Rating_receiver:
    def __init__(self, ip = '192.168.0.3', port = 7100, num_emotion = 9, num_degree = 4):
        self.receive_address = ip, port
        self.num_emotion = num_emotion
        self.num_degree = num_degree
        self.choice = targetlist[0]
        self.degree = degreelist[0]
        self.count = 0
        
     
    def spawn(self):
        print"Server Created."
        self.receiveServer = OSC.OSCServer(self.receive_address) # create a serve to receive OSC from the tablet
        self.receiveServer.addDefaultHandlers()
        
    def initilisation_handler(self, addr, tags, stuff, source):
        global sm, pf, uid, un, run, ns,v
        pf = stuff[0]
        uid = stuff[1]
        un = stuff[2]
        sm = stuff[3]
        run = int(stuff[4])
        ns = stuff[5]
        v = sonmod.parvecunmap(sonmod.parspecs[sm], 
                   sonmod.parspecs[sm]['default'].tolist())
        print stuff

    # Emotion (Happy, surprised,..) change 
    def emotion_handler(self, addr, tags, stuff, source):
        self.choice = targetlist[stuff[0]]
        print stuff[0]
        print self.choice
    # Degree (Weak,average,..) change 
    def degree_handler(self, addr, tags, stuff, source):
        self.degree = degreelist[stuff[0]]
        print self.degree

    # Play the sound based on the parameters
    def play_handler(self, addr, tags, stuff, source):
        global v, sm
        print "inside play event "
        v = mutate(v, 0.4)
        #playfn(0)
        sonmod.playevent(sm, v)
        

    # Save the sound parameter, reset the settings and move to next sound rating
    def next_handler(self, addr, tags, stuff, source):        
        global v, data
        data.append([uid, sm, run, 
                     self.count, time.time()-start_time, v, 
                     self.choice, self.degree])
        reset_settings()
        v = mutate(v, 0.4)
        sonmod.playevent(sm, v)
        self.count += 1
        print "next next handler"

    # Save the sound in a csv file 
    def save_handler(self, addr, tags, stuff, source):
        global data, un, run
        print "completed. thanks."
        fname = "../data/%s-%s-run%d" % (un, sm, run) # All 3 needs to be replaced. 
#         savedata(data, fname)
        df = pandas.DataFrame(data, 
            columns=['uid', 'snd', 'run', 'step', 'time', 'parvec', 'emotion', 'intensity'])
        df.to_csv(fname + time.strftime("-%Y%m%d-%H%M%S") + ".csv", index=False)
        data.append([userid.value, sm, run, 
                     self.count, time.time()-start_time, v, 
                     self.choice, self.degree])
        print "data saved to " + fname
        print "Please inform the operator."
        self.count = 0 # Reset counter. 

    
    def add_handler(self):
        self.receiveServer.addMsgHandler("/play", self.play_handler)
        self.receiveServer.addMsgHandler("/next", self.next_handler)
        self.receiveServer.addMsgHandler("/emo", self.emotion_handler)
        self.receiveServer.addMsgHandler("/degree", self.degree_handler)
        self.receiveServer.addMsgHandler("/save", self.save_handler)
        self.receiveServer.addMsgHandler("/init", self.initilisation_handler)
            
    def print_registered_func(self):
        for addr in self.receiveServer.getOSCAddressSpace():
            print addr
            
    def start(self):
        # Start OSCServer
        print "\nStarting OSCServer."
        self.emorating_oscServer = threading.Thread(target = self.receiveServer.serve_forever)
        self.emorating_oscServer.start()
        print "\nOSCServer established."
        
    def stop(self):
        # Close the OSC server
        print "\nClosing OSCServer."
        self.receiveServer.close()
        print "Waiting for Server-thread to finish"
        try:
            self.emorating_oscServer.join() ##!!!
            print "Done"
        except AttributeError:
            print "Done"



In [15]:
init_receiver = Rating_receiver (ip = ipAddr, port = 8022,num_emotion = 9, num_degree = 4)
init_receiver.spawn()
init_receiver.add_handler()
init_receiver.start()

Server Created.

Starting OSCServer.

OSCServer established.
inside play event 
[ 0.50834745  0.          1.          0.          0.29296585  0.2372114
  0.68154     0.20414917  0.          1.          0.48903919]
size is 11
inside play event 
[ 1.          0.14207283  0.40961758  0.          0.32681093  0.37403383
  0.83922805  0.56726755  0.08822366  0.59970006  0.83852835]
size is 11
2
angry
inside play event 
[ 0.49091815  0.19801593  0.47318736  0.27820881  0.30302591  1.
  0.55653608  0.          0.          0.59231985  0.56422429]
size is 11
3
afraid
inside play event 
[ 0.46619761  0.61263421  0.43558594  0.77385528  0.70360285  1.
  0.03009239  0.          0.35242703  0.52212061  0.33532537]
size is 11
inside play event 
[ 0.06425329  1.          0.67411146  0.4539695   1.          0.87485629
  0.          0.          0.          1.          0.30376911]
size is 11
inside play event 
[ 0.          0.60240138  0.48178313  0.6925226   1.          0.93610768
  0.          0.      

In [14]:
try:
    init_receiver.stop()
except:
    print "OK"


Closing OSCServer.
Waiting for Server-thread to finish
Done


Exception in thread Thread-6:
Traceback (most recent call last):
  File "/Users/sozen/anaconda/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/Users/sozen/anaconda/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/Users/sozen/anaconda/lib/python2.7/site-packages/OSC.py", line 1816, in serve_forever
    self.handle_request()	# this times-out when no data arrives.
  File "/Users/sozen/anaconda/lib/python2.7/SocketServer.py", line 271, in handle_request
    fd_sets = _eintr_retry(select.select, [self], [], [], timeout)
  File "/Users/sozen/anaconda/lib/python2.7/SocketServer.py", line 150, in _eintr_retry
    return func(*args)
error: (9, 'Bad file descriptor')

